In [1]:
%load_ext sql
import json
import os
import cx_Oracle
cx_Oracle.init_oracle_client("C:\\app\\instantclient_21_11")


In [2]:
%%html
<style>
td {text-align: left !important; valign: left !important;}
</style>

# Database migration with the Liberatii Data Platform

The Liberatii Data Platform virtualizes a PostgreSQL database to enable Oracle applications to be migrated to PostgreSQL *without modification*. This notebook contains a tutorial and demonstration of the deployment and use of the Liberatii Data Platform to migrate and test an example Oracle database.

## Architecture

The Liberatii Data Platform uses the Liberatii Gateway to virtualize Azure Databases. This allows
applications built for Oracle to use these databases with **no code changes** and
**no new tests** required.

<img src="Architecture.png" style="margin:auto" alt="Diagram of Liberatii Platform Architecture" title="Architecture"/>

### Change-Data-Capture and Parallel Test Execution

Production workloads can be tested against up-to-date Azure replicas of the Oracle Database
virtualised through Liberatii Gateway using the Change-Data-Capture (CDC) and Parallel Test Execution
features of the Liberatii Data Platform.

<img src="Replay.png" style="margin:auto" alt="Diagram of Workload Replay" title="Replay"/>

This technique provides full confidence for a successful migration **without additional tests**
before any applications are re-targeted to the Liberatii Gateway.

## Demonstration

This notebook uses the
[Oracle H.R. schema](https://docs.oracle.com/cd/E11882_01/server.112/e40540/tablecls.htm#CNCPT111)
to demonstrate the migration of the data and execution of existing Oracle PL/SQL code via
the Liberatii Gateway on a PostgreSQL database.

The H.R. schema provides a basis for queries that use several Oracle-specific features that
are virtualized through the Liberatii Gateway:

<img src="Schema.gif" style="margin:auto" alt="Diagram of the HR Schema" title="Schema"/>

## Migrating an application

This demonstration will migrate the schema and correspdoning SQL with Liberatii Data Platform though the following steps:

1. **Deployment**<br/>
   The deployment of an Azure Managed Application to provide the Liberatii Data Platform
   
3. **Migration**<br/>
   The migration of the schema and data from the Oracle database to PostgreSQL via the Liberatii Gateway
   
5. **Synchronisation**<br/>
   Setup of a Change Data Capture pipeline to synchronize the Oracle and PostgreSQL databases
   
7. **Replay testing**<br/>
   Testing of an Oracle Workload Replay for **performance** and **correctness** against both the Oracle and PostgreSQL databases to verify proper
   operation
   
9. **Switch over**<br/>
   Replacing the database driver and retargeting the application to use the Liberatii Gateway

# Set up

The following cell contains the connection information for the Oracle (Source) and PostgreSQL (Target) databases. It is initially set up to migrate the Oracle demo H.R. schema and expects the given users to exist with the required permissions on the specified databases.

This Notebook uses the SQL extension so connections the databases can be queried directly. The following cells check the connections to Oracle and PostgreSQL are working:

In [3]:
## Hostnames of the databases
PG_HOST='demo-g3rqzbqudztzk-pgsql.postgres.database.azure.com'
ORACLE_HOST='192.168.1.6'
LGW_HOST='192.168.1.4'

## Connection data
DB='PDBORCL'
USER='HR'
PSWD='hr'
ORACLE_PORT=1521
PG_PORT=5432
LGW_PORT=5432
PLATFORM='migration'

## Connection strings (derived from the above connection data)
ORACLE_CONN_STR=f'(DESCRIPTION=(ADDRESS=(PROTOCOL=TCP)(HOST={ORACLE_HOST})(PORT={ORACLE_PORT}))(CONNECT_DATA=(SERVICE_NAME={DB})))'
PG=f'postgresql://{USER}:{PSWD}@{PG_HOST}:{PG_PORT}/{DB}'
ORACLE=f'oracle://{USER}:{PSWD}@{ORACLE_CONN_STR}'
LGW=f'postgresql://{USER}:{PSWD}@{LGW_HOST}:{LGW_PORT}/{DB}'
API_PREFIX=f"http://{LGW_HOST}:3000"

print(f"""
Connection settings:

    PostgreSQL:             
      {PG}
    Oracle:                 
      {ORACLE}
    Liberatii Data Platform:
      {API_PREFIX}/api
    LGW:
      {LGW}
""")


Connection settings:

    PostgreSQL:             
      postgresql://HR:hr@demo-g3rqzbqudztzk-pgsql.postgres.database.azure.com:5432/PDBORCL
    Oracle:                 
      oracle://HR:hr@(DESCRIPTION=(ADDRESS=(PROTOCOL=TCP)(HOST=192.168.1.6)(PORT=1521))(CONNECT_DATA=(SERVICE_NAME=PDBORCL)))
    Liberatii Data Platform:
      http://192.168.1.4:3000/api
    LGW:
      postgresql://HR:hr@192.168.1.4:5432/PDBORCL



In [4]:
%%sql {ORACLE}
-- Find version information for the Oracle database
select banner FROM v$version

0 rows affected.


banner
Oracle Database 12c Enterprise Edition Release 12.1.0.2.0 - 64bit Production
PL/SQL Release 12.1.0.2.0 - Production
CORE 12.1.0.2.0 Production
TNS for 64-bit Windows: Version 12.1.0.2.0 - Production
NLSRTL Version 12.1.0.2.0 - Production


In [5]:
%%sql {PG}
-- Find version information for the PostgreSQL database
select version()

1 rows affected.


version
"PostgreSQL 14.8 on x86_64-pc-linux-gnu, compiled by gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0, 64-bit"


On LGW we run Oracle SQL query, but it returns PostgreSQL version:

In [6]:
%%sql {LGW}
-- Find version information for the PostgreSQL database via Liberatii Gateway
select banner FROM v$version

1 rows affected.


BANNER
"PostgreSQL 14.8 on x86_64-pc-linux-gnu, compiled by gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0, 64-bit"


# Migration

The following section shows the migration of the schema and data from the Oracle database to PostgreSQL. This uses the Liberatii Data Platform to perform the following steps:

1. Configuration, connection setup and initialisation
3. Schema Migration
4. Data Migration
5. Testing and verification

All of these steps **can be performed in parallel** across multiple threads and connections to cope with large volumes of data.

## Configuration

The following cells will construct the required connection information. Each connection (Oracle, PostgreSQL and the Liberatii Gateway) will be created in turn.

### Oracle

In [7]:
!curl -s {API_PREFIX}/connection -H "Content-Type: application/json" \
   -d "{{\"type\":\"Oracle\",\"connectionString\":\"{ORACLE_CONN_STR}\",\"user\":\"{USER}\",\"password\":\"{PSWD}\",\"id\":1}}"

{"type":"Oracle","connectionString":"(DESCRIPTION=(ADDRESS=(PROTOCOL=TCP)(HOST=192.168.1.6)(PORT=1521))(CONNECT_DATA=(SERVICE_NAME=PDBORCL)))","user":"HR","password":"hr","id":1}


### PostgreSQL

In [8]:
!curl -s {API_PREFIX}/connection -H "Content-Type: application/json" \
  -d "{{\"type\":\"PostgreSQL\",\"host\":\"{PG_HOST}\",\"port\":5432,\"database\":\"{DB}\",\"user\":\"{USER}\",\"password\":\"{PSWD}\",\"id\":2}}"

{"type":"PostgreSQL","host":"demo-g3rqzbqudztzk-pgsql.postgres.database.azure.com","port":5432,"database":"PDBORCL","user":"HR","password":"hr","id":2}


### Liberatii Gateway:

In [9]:
!curl -s {API_PREFIX}/connection -H "Content-Type: application/json" \
  -d "{{\"type\":\"LGW\",\"host\":\"{LGW_HOST}\",\"port\":{LGW_PORT},\"database\":\"{DB}\",\"user\":\"{USER}\",\"password\":\"{PSWD}\",\"id\":3}}"

{"type":"LGW","host":"192.168.1.4","port":5432,"database":"PDBORCL","user":"HR","password":"hr","id":3}


### Configuration Parameters

The following configuration parameters are used for this demonstration. Please check the reference document to get all the available parameters.

* `dataOnePass`<br/>
  Don't use staging files or tables, just transfer all the data from the source to the target
* `user`<br/>
  List of schemas to transfer between the sources and target target
* `verbose`<br/>
  Log verbosity
* `eraseOnInit`<br/>
  Delete everything from the schema in the init stage, this is useful if we often restart the migration.

In [10]:
result = !curl -s {API_PREFIX}/config -H "Content-Type: application/json" \
  -d "{{\"dataOnePass\": true, \"users\":[\"{USER}\"], \"verbose\":4, \"eraseOnInit\":true}}"
print(json.dumps(json.loads(result[0]), indent=True))

{
 "message": "Config has been set successfully",
 "config": {
  "dataOnePass": true,
  "verbose": 4,
  "useCopy": true,
  "dataIterations": -1,
  "useWrapper": true,
  "useNative": false,
  "useUnlogged": false,
  "stat": true,
  "statDB": false,
  "rowsBuf": 1000,
  "lightCheck": false,
  "dataChunkSize": -1,
  "bigTablesFirst": true,
  "debReverseOrder": false,
  "cli": true,
  "rmStagingFiles": true,
  "parTables": true,
  "hashType": "murmur",
  "simulateUnsupportedTypes": true,
  "blobStreams": true,
  "clobStreams": false,
  "blobImmed": false,
  "clobImmed": false,
  "dumpCopy": false,
  "dryRun": false,
  "noBlobs": false,
  "idCol": "rowid",
  "idColByTable": {},
  "activeSqlFiles": [],
  "numTries": 10,
  "maxWait": 60000,
  "ignoreTrim": true,
  "removeLastFetchFirst": false,
  "checkMetaData": true,
  "users": [
   "HR"
  ],
  "linkedServers": [],
  "stages": {},
  "workloadsFiles": "",
  "eraseOnInit": true
 }
}


### Initialisation

This is an initialisation operation. The framework runs simple assessments and stores the results in the target database. The operation is asynchronous: it just schedules the operation and exists immediately.

In [11]:
!curl -s {API_PREFIX}/operation -H "Content-Type: application/json" -d "{{ \"oracle\": 1, \"lgw\": 3, \"stage\": \"init\" }}"

{"message":"Config has been set successfully.","config":{"title":"init","status":"Running","messages":[],"progress":0}}


Using this operation we can get the current state of the currently running operation:

In [12]:
!curl -s {API_PREFIX}/operation?pager=1 -H "Content-Type: application/json"

{"title":"init","status":"Running","messages":[],"progress":0}


And the next operation is the same as the previous one, except it waits until the current operation ends.

In [13]:
!curl -s -X POST {API_PREFIX}/operation/wait -H "Content-Type: application/json"

{"title":"init","status":"Running","messages":[{"level":"Info","message":"\n^g^+DONE! ðŸ‘�^:\n"},{"level":"Info","message":[["Type","I","Total"],["TABLE","7","7"],["PROCEDURE","2","2"],["TRIGGER","2","2"],["SEQUENCE","3","3"],["INDEX","11","11"],["VIEW","1","1"],["^+Total:^:","26","^+26^"]]}],"progress":0}


## Schema migration

After `init` stage is complete we have all the necessary information for migrating the database schema.

The migration metadata is stored in the PostgreSQL database allowing us to run various queries on it. For example, we can query the number of objects of each type by running with the query:


In [14]:
%%sql {PG}
select count(*), type, stage, error from dbt.migration_objects group by type, stage, error

6 rows affected.


count,type,stage,error
11,INDEX,I,None
2,PROCEDURE,I,None
3,SEQUENCE,I,None
7,TABLE,I,None
1,VIEW,I,None
2,TRIGGER,I,None


If there were any errors during the execution of any stage, they will be displayed in `error` stage. The errors can be manually fixed by modifying `ddl1`, `ddl2` columns of `dbt.migration_objects` or by adding some runtime objects:

In [15]:
%%sql {PG}
select ddl1, ddl2 from dbt.migration_objects where type = 'TABLE' limit 1

1 rows affected.


ddl1,ddl2
"CREATE TABLE ""HR"".""EMPLOYEES"" ( ""EMPLOYEE_ID"" NUMBER(6,0), ""FIRST_NAME"" VARCHAR2(20), ""LAST_NAME"" VARCHAR2(25) CONSTRAINT ""EMP_LAST_NAME_NN"" NOT NULL ENABLE, ""EMAIL"" VARCHAR2(25) CONSTRAINT ""EMP_EMAIL_NN"" NOT NULL ENABLE, ""PHONE_NUMBER"" VARCHAR2(20), ""HIRE_DATE"" DATE CONSTRAINT ""EMP_HIRE_DATE_NN"" NOT NULL ENABLE, ""JOB_ID"" VARCHAR2(10) CONSTRAINT ""EMP_JOB_NN"" NOT NULL ENABLE, ""SALARY"" NUMBER(8,2), ""COMMISSION_PCT"" NUMBER(2,2), ""MANAGER_ID"" NUMBER(6,0), ""DEPARTMENT_ID"" NUMBER(4,0) )","ALTER TABLE ""HR"".""EMPLOYEES"" ADD CONSTRAINT ""EMP_SALARY_MIN"" CHECK (salary > 0) ENABLE; ALTER TABLE ""HR"".""EMPLOYEES"" ADD CONSTRAINT ""EMP_EMAIL_UK"" UNIQUE (""EMAIL"") USING INDEX ENABLE; ALTER TABLE ""HR"".""EMPLOYEES"" ADD CONSTRAINT ""EMP_EMP_ID_PK"" PRIMARY KEY (""EMPLOYEE_ID"") USING INDEX ENABLE; ALTER TABLE ""HR"".""EMPLOYEES"" ADD CONSTRAINT ""EMP_DEPT_FK"" FOREIGN KEY (""DEPARTMENT_ID"") REFERENCES ""HR"".""DEPARTMENTS"" (""DEPARTMENT_ID"") ENABLE; ALTER TABLE ""HR"".""EMPLOYEES"" ADD CONSTRAINT ""EMP_JOB_FK"" FOREIGN KEY (""JOB_ID"") REFERENCES ""HR"".""JOBS"" (""JOB_ID"") ENABLE; ALTER TABLE ""HR"".""EMPLOYEES"" ADD CONSTRAINT ""EMP_MANAGER_FK"" FOREIGN KEY (""MANAGER_ID"") REFERENCES ""HR"".""EMPLOYEES"" (""EMPLOYEE_ID"") ENABLE;"


When errors are fixed we don't need to run the whole migration from the beginning. It's enough to reset only the problematic objects' `stage` field and restart the stage. Only those objects migration will be re-run.

The schema migration doesn't migrate constraints, indexes and triggers. They are migrated in `constraints` stage. This is done to make data migration run faster. The Liberatii Data Platform can also execute **multiple parallel instances** of each operation to signficantly improve transfer speeds. It is even possible to perform **parallel migrations for single tables** by splitting them.

The schema migration is started with same parameters as with `init` stage, but with `schema` as the stage name.

In [16]:
!curl -s {API_PREFIX}/operation -H "Content-Type: application/json" -d "{{ \"oracle\": 1, \"lgw\": 3, \"stage\": \"schema\" }}"

{"message":"Config has been set successfully.","config":{"title":"init","status":"Running","messages":[],"progress":0}}


Again, waiting until this task is finished:


In [17]:
!curl -s -X POST {API_PREFIX}/operation/wait -H "Content-Type: application/json"

{"title":"schema","status":"Running","messages":[{"level":"Info","message":"^+ðŸ‘�^: ^gDONE!^: translating SEQUENCE \"HR\".\"LOCATIONS_SEQ\"\n"},{"level":"Info","message":"^+ðŸ‘�^: ^gDONE!^: translating TABLE \"HR\".\"EMPLOYEES\"\n"},{"level":"Info","message":"^+ðŸ‘�^: ^gDONE!^: translating TABLE \"HR\".\"LOCATIONS\"\n"},{"level":"Info","message":"^+ðŸ‘�^: ^gDONE!^: translating TABLE \"HR\".\"REGIONS\"\n"},{"level":"Info","message":"^+ðŸ‘�^: ^gDONE!^: translating TABLE \"HR\".\"DEPARTMENTS\"\n"},{"level":"Info","message":"^+ðŸ‘�^: ^gDONE!^: translating SEQUENCE \"HR\".\"DEPARTMENTS_SEQ\"\n"},{"level":"Info","message":"^+ðŸ‘�^: ^gDONE!^: translating SEQUENCE \"HR\".\"EMPLOYEES_SEQ\"\n"},{"level":"Info","message":"^+ðŸ‘�^: ^gDONE!^: translating PROCEDURE \"HR\".\"SECURE_DML\"\n"},{"level":"Info","message":"^+ðŸ‘�^: ^gDONE!^: translating TABLE \"HR\".\"JOB_HISTORY\"\n"},{"level":"Info","message":"^+ðŸ‘�^: ^gDONE!^: translating PROCEDURE \"HR\".\"ADD_JOB_HISTORY\"\n"},{"level":"Info","mess

In the `init` stage the framework also queries the sizes of each table. Using them we can get the most optimal strategy for copying the data.

## Data

So let's see the sizes:


In [18]:
%%sql {PG}
select name, pg_size_pretty(data_size), stage, error from dbt.migration_objects where type = 'TABLE' order by data_size desc

7 rows affected.


name,pg_size_pretty,stage,error
JOB_HISTORY,64 kB,D,None
REGIONS,64 kB,D,None
DEPARTMENTS,64 kB,D,None
LOCATIONS,64 kB,D,None
EMPLOYEES,64 kB,D,None
JOBS,64 kB,D,None
COUNTRIES,0 bytes,D,None



By analysing the sizes for different tables we can apply different strategies for different tables. But size the sizes are quire small for this schema we just use the default strategy everywhere.

Now starting the data migration operation, as all the operations before, but with `data` as `stage` field value:


In [19]:
!curl -s {API_PREFIX}/operation -H "Content-Type: application/json" -d "{{ \"oracle\": 1, \"lgw\": 3, \"stage\": \"data\" }}"

{"message":"Config has been set successfully.","config":{"title":"init","status":"Running","messages":[],"progress":0}}


Awaiting the operation to be completed:

In [20]:
!curl -s -X POST {API_PREFIX}/operation/wait -H "Content-Type: application/json"

{"title":"data","status":"Running","messages":[{"level":"Info","message":"^+ðŸ‘�^: ^gDONE!^: migrating data \"HR\".\"JOB_HISTORY\"in 0.154s(exec=0.1142s,copy-stmt=0.0014s,rows=0s,write=0.0014s,set-state=0.0301s)\n"},{"level":"Info","message":"^+ðŸ‘�^: ^gDONE!^: migrating data \"HR\".\"LOCATIONS\"in 0.113s(exec=0.073s,copy-stmt=0.0002s,rows=0s,write=0.0002s,set-state=0.032s)\n"},{"level":"Info","message":"^+ðŸ‘�^: ^gDONE!^: migrating data \"HR\".\"REGIONS\"in 0.114s(exec=0.0753s,copy-stmt=0.0001s,rows=0s,write=0.0001s,set-state=0.0313s)\n"},{"level":"Info","message":"^+ðŸ‘�^: ^gDONE!^: migrating data \"HR\".\"DEPARTMENTS\"in 0.106s(exec=0.0669s,copy-stmt=0.0002s,rows=0s,write=0.0002s,set-state=0.03s)\n"},{"level":"Info","message":"^+ðŸ‘�^: ^gDONE!^: migrating data \"HR\".\"EMPLOYEES\"in 0.12s(exec=0.0724s,copy-stmt=0.0028s,rows=0s,write=0.0028s,set-state=0.0316s)\n"},{"level":"Info","message":"^+ðŸ‘�^: ^gDONE!^: migrating data \"HR\".\"JOBS\"in 0.106s(exec=0.0664s,copy-stmt=0.0002s,rows

## Constraints
This stage adds constraints, indexes and triggers to the data we've moved in the previous step:


In [21]:
!curl -s {API_PREFIX}/operation -H 'Content-Type: application/json' -d '{{ "oracle": 1, "lgw": 3, "stage": "constraints" }}'

{"statusCode":404,"message":"Connection undefined does not exist or is not an Oracle connection!","error":"Not Found"}


Awaiting the operation to be completed:

In [22]:
!curl -s -X POST {API_PREFIX}/operation/wait -H 'Content-Type: application/json'

{"title":"data","status":"Running","messages":[{"level":"Info","message":"^+ðŸ‘�^: ^gDONE!^: migrating data \"HR\".\"JOB_HISTORY\"in 0.154s(exec=0.1142s,copy-stmt=0.0014s,rows=0s,write=0.0014s,set-state=0.0301s)\n"},{"level":"Info","message":"^+ðŸ‘�^: ^gDONE!^: migrating data \"HR\".\"LOCATIONS\"in 0.113s(exec=0.073s,copy-stmt=0.0002s,rows=0s,write=0.0002s,set-state=0.032s)\n"},{"level":"Info","message":"^+ðŸ‘�^: ^gDONE!^: migrating data \"HR\".\"REGIONS\"in 0.114s(exec=0.0753s,copy-stmt=0.0001s,rows=0s,write=0.0001s,set-state=0.0313s)\n"},{"level":"Info","message":"^+ðŸ‘�^: ^gDONE!^: migrating data \"HR\".\"DEPARTMENTS\"in 0.106s(exec=0.0669s,copy-stmt=0.0002s,rows=0s,write=0.0002s,set-state=0.03s)\n"},{"level":"Info","message":"^+ðŸ‘�^: ^gDONE!^: migrating data \"HR\".\"EMPLOYEES\"in 0.12s(exec=0.0724s,copy-stmt=0.0028s,rows=0s,write=0.0028s,set-state=0.0316s)\n"},{"level":"Info","message":"^+ðŸ‘�^: ^gDONE!^: migrating data \"HR\".\"JOBS\"in 0.106s(exec=0.0664s,copy-stmt=0.0002s,rows

## Verification

With the data in place we can verify the migration across both databases to ensure they are the same. This is done by comparing hash sums run across data in each column:

In [23]:
!curl -s {API_PREFIX}/operation -H "Content-Type: application/json" -d "{{ \"oracle\": 1, \"lgw\": 3, \"stage\": \"check\" }}"

{"message":"Config has been set successfully.","config":{"title":"init","status":"Running","messages":[],"progress":0}}


In [24]:
!curl -s -X POST {API_PREFIX}/operation/wait -H "Content-Type: application/json"

{"title":"check","status":"Running","messages":[{"level":"Info","message":"ðŸ¤ž checking \"HR\".\"JOB_HISTORY\" (0.06M)...\n"},{"level":"Info","message":"^+ðŸ‘�^: ^gDONE!^: checking data \"HR\".\"JOB_HISTORY\"\n"},{"level":"Info","message":"ðŸ¤ž checking \"HR\".\"LOCATIONS\" (0.06M)...\n"},{"level":"Info","message":"^+ðŸ‘�^: ^gDONE!^: checking data \"HR\".\"LOCATIONS\"\n"},{"level":"Info","message":"ðŸ¤ž checking \"HR\".\"REGIONS\" (0.06M)...\n"},{"level":"Info","message":"^+ðŸ‘�^: ^gDONE!^: checking data \"HR\".\"REGIONS\"\n"},{"level":"Info","message":"ðŸ¤ž checking \"HR\".\"DEPARTMENTS\" (0.06M)...\n"},{"level":"Info","message":"^+ðŸ‘�^: ^gDONE!^: checking data \"HR\".\"DEPARTMENTS\"\n"},{"level":"Info","message":"ðŸ¤ž checking \"HR\".\"EMPLOYEES\" (0.06M)...\n"},{"level":"Info","message":"^+ðŸ‘�^: ^gDONE!^: checking data \"HR\".\"EMPLOYEES\"\n"},{"level":"Info","message":"ðŸ¤ž checking \"HR\".\"JOBS\" (0.06M)...\n"},{"level":"Info","message":"^+ðŸ‘�^: ^gDONE!^: checking data \"HR\"

In [25]:
%%sql {PG}

select stage, type from dbt.migration_objects group by stage, type

6 rows affected.


stage,type
K,TABLE
D,PROCEDURE
I,TRIGGER
D,SEQUENCE
D,VIEW
I,INDEX


This is it, now the whole database is migrated. Please see the guide document for the information about different migration options.

## Running queries

Let's now run a few queries in the both databases to see the results are identical. There is a `checksql` stage to do this automatically, but for the demo purposes we do this manually.

So first we check the databases are indeed Oracle and PostgreSQL:

In [26]:
%%sql {ORACLE}
select banner from v$version

0 rows affected.


banner
Oracle Database 12c Enterprise Edition Release 12.1.0.2.0 - 64bit Production
PL/SQL Release 12.1.0.2.0 - Production
CORE 12.1.0.2.0 Production
TNS for 64-bit Windows: Version 12.1.0.2.0 - Production
NLSRTL Version 12.1.0.2.0 - Production


In [27]:
%%sql {LGW}
select banner from v$version

1 rows affected.


BANNER
"PostgreSQL 14.8 on x86_64-pc-linux-gnu, compiled by gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0, 64-bit"


The output value on PostgreSQL can be configured to output exactly the same like Oracle if the application uses the data for some business logic selection.

Now let's run the same query in both databases. This query contains a number of Oracle-specific features:

* Concatenation using the double-pipe (||) operator
* The `NVL` and `DECODE` functions
* The join-plus operator and implicit joins

In [28]:
QUERY="""
SELECT first_name  || ' ' ||  last_name full_name,
        salary +
        NVL  (commission_pct, 0) sal_com,
        DECODE (NVL(e.department_id, -3),
                    60, d.department_name,
                    50, d.department_name,
                    30, d.department_name,
                    -3, 'UNKNOWN',
               'OTHER') dep,
        TO_CHAR(e.hire_date) hire_date
FROM employees e, departments d
WHERE d.department_id(+) = e.department_id
        AND e.last_name like 'G%'
ORDER BY first_name  ||  ' '  ||  last_name
"""

First running it on Oracle:

In [29]:
%%sql {ORACLE} 

{QUERY}

0 rows affected.


full_name,sal_com,dep,hire_date
Danielle Greene,9500.15,OTHER,19-MAR-07
Douglas Grant,2600,Shipping,13-JAN-08
Girard Geoni,2800,Shipping,03-FEB-08
Ki Gee,2400,Shipping,12-DEC-07
Kimberely Grant,7000.15,UNKNOWN,24-MAY-07
Nancy Greenberg,12008,OTHER,17-AUG-02
Timothy Gates,2900,Shipping,11-JUL-06
William Gietz,8300,OTHER,07-JUN-02


And now running absolutely the same query but on PostgreSQL via Liberatii Gateway:

In [30]:
%%sql {LGW}

{QUERY}

8 rows affected.


FULL_NAME,SAL_COM,DEP,HIRE_DATE
Danielle Greene,9500.15,OTHER,19-MAR-07
Douglas Grant,2600.00,Shipping,13-JAN-08
Girard Geoni,2800.00,Shipping,03-FEB-08
Ki Gee,2400.00,Shipping,12-DEC-07
Kimberely Grant,7000.15,UNKNOWN,24-MAY-07
Nancy Greenberg,12008.00,OTHER,17-AUG-02
Timothy Gates,2900.00,Shipping,11-JUL-06
William Gietz,8300.00,OTHER,07-JUN-02


The results are identical. 

The next steps are:

* Synchronise the databases using CDC so we avoid downtimes on switchover
* Replay workloads on a sandbox Oracle database and PostgreSQL via Liberatii Gateway, for testing correctness and performance
